In [1]:
import stanza

In [2]:
from tqdm import tqdm

In [3]:
nlp = stanza.Pipeline(lang='en', processors='tokenize, lemma ,pos')

2023-04-07 21:31:56 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-04-07 21:31:57 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |

2023-04-07 21:31:57 INFO: Using device: cuda
2023-04-07 21:31:57 INFO: Loading: tokenize
2023-04-07 21:32:01 INFO: Loading: pos
2023-04-07 21:32:01 INFO: Loading: lemma
2023-04-07 21:32:01 INFO: Done loading processors!


In [4]:
import pandas as pd

In [5]:
ingredient_df = pd.read_csv("temp.csv")

In [6]:
ingredient_list = list(ingredient_df.ingredient)

In [7]:
stanza_res=[]

In [8]:
for i in tqdm(ingredient_list):
    stanza_res.append(nlp(i))

100%|██████████| 14901/14901 [05:33<00:00, 44.65it/s]


In [10]:
final_words = []

In [11]:
i = 0
for res in tqdm(stanza_res):
    noun_list = []
    if len(res.sentences) > 1:
        print(i)
    
    else:
        for sentence in res.sentences:
            noun_list = []
            len_word = len(sentence.words)
            j = 0
            for word in sentence.words:
                if (len_word - 1) == j:
                    if word.upos == "NOUN":
                        noun_list.append(word.lemma)
                    else:
                        noun_list = []
                else:
                    if word.upos == "NOUN":
                        noun_list.append(word.text)
                    else:
                        noun_list = []
                j = j + 1
                    
    final_words.append(" ".join(noun_list))
    i = i + 1

100%|██████████| 14901/14901 [00:00<00:00, 295556.78it/s]

6860
6861
6862
9603
10054
10055
11795


In [19]:
pd.concat([ingredient_df, pd.DataFrame(final_words)], axis=1).to_csv("Foo.csv")

In [12]:
ingredient_list[0]

'1% fat buttermilk'

In [14]:
doc = nlp(ingredient_list[0])

In [28]:
doc.sentences[0]

TypeError: 'Sentence' object is not subscriptable

In [20]:
for sentence in doc.sentences:
    for word in sentence.words:
        if word.upos == "NOUN":
            print(word.text)

buttermilk


In [29]:
len(doc.sentences)

1

In [ ]:
def extract_adjectives_with_nouns(review_text, nlp_stanza):
    """
    Extract the adjectives and the nouns they are describing

    Args:
        review_text (str): review text
    
    Returns:
        adj_noun_pairs (list): list of adjective-noun pairs
    """

    ## extract all the adjectives and the nouns they are describing
    doc = nlp_stanza(review_text)
    adj_noun_pairs = []
    
    for sentence in doc.sentences:
        
        for word in sentence.words:
            if word.upos == "NOUN":
                new_list.append(word)

    return adj_noun_pairs

In [ ]:
doc = nlp('Barack Obama was born in Hawaii.')
print(*[f'word: {word.text}\tupos: {word.upos}\txpos: {word.xpos}\tfeats: {word.feats if word.feats else "_"}' for sent in doc.sentences for word in sent.words], sep='\n')

In [6]:
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

In [7]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5"
model = hub.load(module_url)
print ("module %s loaded" % module_url)

2023-04-07 19:50:15.854452: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-07 19:50:16.139046: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-07 19:50:16.139456: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-07 19:50:16.140675: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild

module https://tfhub.dev/google/universal-sentence-encoder-large/5 loaded


In [8]:
def embed(input):
    return model(input)

In [9]:
def run_and_plot(messages_):
    message_embeddings_ = embed(messages_)
    corr = np.inner(message_embeddings_, message_embeddings_)
    return corr, messages_, message_embeddings_